In [1]:
# First 5 weeks & rewards in 19 Audience only 

import pandas as pd
import datetime
import numpy as np
import os
import glob
import gc

def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)
            
os.getcwd()

# Note how to difine the new sign ups

'/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Projection_est_in_Q3'

In [2]:
set_SOTF_number=pd.ExcelFile("./SOTF/Store List Report 20190910  09.10.19 417PM Dom edited BR .xlsx")
# set_SOTF_number.sheet_names
set_SOTF_number=set_SOTF_number.parse("SOTF pre 11-15-19 (462)",dtype=str)
set_SOTF_number=set_SOTF_number[(set_SOTF_number['SOTF']=="Yes") & (set_SOTF_number['Open Date']<="2019-09-07")]
set_SOTF_number=set(set_SOTF_number['Store'].unique().tolist())
len(set_SOTF_number)

446

In [3]:
new_sign_ups_after_2019Q3_end=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/All Rewards Members 2018-02-04 - 2019-05-04.zip",
                                             compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','sign_up_date'])
new_sign_ups_after_2019Q3_end=new_sign_ups_after_2019Q3_end[new_sign_ups_after_2019Q3_end['sign_up_date']>="2018-08-05"] # First of 2018Q3
master_file_gap=pd.read_table("/home/jian/BigLots/New_Sing_Ups_2018_Fiscal_Year/MediaStorm Rewards Master P4 2019 - no transaction info.zip",
                                             compression="zip",dtype=str,sep="|",usecols=['customer_id_hashed','sign_up_date'])
new_sign_ups_after_2019Q3_end=master_file_gap.append(new_sign_ups_after_2019Q3_end).drop_duplicates()
del master_file_gap
###

all_master_files_after_201906=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
all_master_files_after_201906=[x for x in all_master_files_after_201906 if ("aster" in x) & (".txt" in x)]
all_master_files_after_201906=[x for x in all_master_files_after_201906 if x.split("ts/2019_by_weeks/MediaStorm_")[1][:10]>"2019-06-01"]
df_master_all_later_after_201906=pd.DataFrame()
for file in all_master_files_after_201906:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','sign_up_date'])
    df_master_all_later_after_201906=df_master_all_later_after_201906.append(df)

new_sign_ups_after_2019Q3_end=new_sign_ups_after_2019Q3_end.append(df_master_all_later_after_201906).drop_duplicates()
del df_master_all_later_after_201906


gc.collect()
print("new_sign_ups_after_2019Q3_end.shape",new_sign_ups_after_2019Q3_end.shape)
print("new_sign_ups_after_2019Q3_end['customer_id_hashed'].nunique()",new_sign_ups_after_2019Q3_end['customer_id_hashed'].nunique())

new_sign_ups_after_2019Q3_end.shape (7623850, 2)
new_sign_ups_after_2019Q3_end['customer_id_hashed'].nunique() 7623841


In [4]:
dup_ids=new_sign_ups_after_2019Q3_end[new_sign_ups_after_2019Q3_end['customer_id_hashed'].duplicated()]['customer_id_hashed'].tolist()
df=new_sign_ups_after_2019Q3_end[new_sign_ups_after_2019Q3_end['customer_id_hashed'].isin(dup_ids)]
df.shape

(18, 2)

In [5]:
new_sign_ups_after_2019Q3_end=new_sign_ups_after_2019Q3_end.sort_values(['customer_id_hashed','sign_up_date'],ascending=[True,False]).drop_duplicates("customer_id_hashed")
set_new_signed_ids=set(new_sign_ups_after_2019Q3_end['customer_id_hashed'].tolist())
del new_sign_ups_after_2019Q3_end
gc.collect()

35

In [6]:
audience_list_2019Q3="/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2019Q3/upload_files/"
audience_list_2019Q3=glob.glob(audience_list_2019Q3+"*.csv")
len(audience_list_2019Q3)


audience_list_2019Q3_C=[x for x in audience_list_2019Q3 if os.path.basename(x)[0]=="C"]
audience_list_2019Q3_T=[x for x in audience_list_2019Q3 if os.path.basename(x)[0]=="T"]
print(len(audience_list_2019Q3_C),len(audience_list_2019Q3_T))

set_C_2019Q3=set()
for file in audience_list_2019Q3_C:
    df=pd.read_csv(file,dtype=str,usecols=["customer_id_hashed"])
    set_C_2019Q3=set_C_2019Q3.union(set(df['customer_id_hashed'].tolist()))
print(len(set_C_2019Q3))


df_audience_2019Q3=pd.DataFrame()

for file in audience_list_2019Q3_T:
    df=pd.read_csv(file,dtype=str,usecols=['customer_id_hashed','segment'])
    df=df[~df['customer_id_hashed'].isin(set_C_2019Q3)]
    df_audience_2019Q3=df_audience_2019Q3.append(df)
df_audience_2019Q3=df_audience_2019Q3.drop_duplicates()

print(df_audience_2019Q3.shape," | ",df_audience_2019Q3['customer_id_hashed'].nunique())

25 25
542702
(21810554, 2)  |  21810554


In [7]:
# Rolling up removing the quadrant
df_audience_total_count_original=df_audience_2019Q3.groupby("segment")['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"ids_total"})
df_audience_2019Q3['segment_2']=df_audience_2019Q3['segment'].str.replace("Quadrant I_","")
df_audience_2019Q3['segment_2']=df_audience_2019Q3['segment_2'].str.replace("Quadrant II_","")
df_audience_2019Q3['segment_2']=df_audience_2019Q3['segment_2'].str.replace("Quadrant III_","")
df_audience_2019Q3['segment_2']=df_audience_2019Q3['segment_2'].str.replace("Quadrant IV_","")
print(df_audience_2019Q3.shape)

df_audience_total_count=df_audience_2019Q3.groupby("segment_2")['customer_id_hashed'].nunique().to_frame().reset_index().rename(columns={"customer_id_hashed":"ids_total"})

del df_audience_2019Q3['segment']
df_audience_2019Q3=df_audience_2019Q3.rename(columns={"segment_2":"segment"})

print(df_audience_total_count.shape)
df_audience_total_count

(21810554, 3)
(7, 2)


,segment_2,ids_total
0,T_P_H_2019Q3_RFM,4937981
1,T_P_L_2019Q3_RFM,3391401
2,T_P_M_2019Q3_RFM,3515875
3,T_S_H_2019Q3_RFM,1694485
4,T_S_L_2019Q3_RFM,1422640
5,T_S_M_2019Q3_RFM,1361003
6,T_lapsed_18_48_PS_RFM,5487169


In [8]:
sales_file_2019Q3=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
sales_file_2019Q3=[x for x in sales_file_2019Q3 if x.split("MediaStorm_")[1][:10]>="2019-08-05" and x.split("MediaStorm_")[1][:10]<="2019-09-14" and "Daily" in x]
len(sales_file_2019Q3)


6

In [9]:
# First 6 weeks only
sales_file_2018Q3=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
sales_file_2018Q3=[x for x in sales_file_2018Q3 if x.split("MediaStormDailySalesHistory")[1][:8]>"20180804" and x.split("MediaStormDailySalesHistory")[1][:8]<="20180915"]
len(sales_file_2018Q3)

6

In [10]:
df_agg_POS_data_2018Q3_rewards=pd.DataFrame()

i_counter=0
for file in sales_file_2018Q3:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed',
                                                     'item_transaction_units','item_transaction_amt'])
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['item_transaction_units']=df['item_transaction_units'].astype(int)
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    
    df_sales=df.groupby(['location_id','transaction_dt','customer_id_hashed'])['item_transaction_units','item_transaction_amt'].sum().reset_index()
    df_sales=df_sales.rename(columns={"item_transaction_units":"units","item_transaction_amt":"sales"})
    
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans['trans']=1
    df_trans=df_trans.groupby(['location_id','transaction_dt','customer_id_hashed'])['trans'].sum().to_frame().reset_index()
    
    df=pd.merge(df_sales,df_trans,on=["location_id",'transaction_dt','customer_id_hashed'],how="outer")
    df_agg_POS_data_2018Q3_rewards=df_agg_POS_data_2018Q3_rewards.append(df)
    
    print(datetime.datetime.now(),i_counter,df_agg_POS_data_2018Q3_rewards.shape)
    i_counter+=1
gc.collect()

2019-09-18 11:25:48.727100 0 (1190721, 6)
2019-09-18 11:33:34.195734 1 (2414440, 6)
2019-09-18 11:39:30.651773 2 (3646503, 6)
2019-09-18 11:45:02.110368 3 (4894015, 6)
2019-09-18 11:50:40.877220 4 (6144836, 6)
2019-09-18 11:55:45.584474 5 (7315812, 6)


96

In [11]:
df_agg_POS_data_2019Q3_rewards=pd.DataFrame()

i_counter=0
for file in sales_file_2019Q3:
    df=pd.read_table(file,dtype=str,sep="|",usecols=['location_id','transaction_dt','transaction_id','customer_id_hashed',
                                                     'item_transaction_units','item_transaction_amt'])
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['item_transaction_units']=df['item_transaction_units'].astype(int)
    df['item_transaction_amt']=df['item_transaction_amt'].astype(float)
    
    df_sales=df.groupby(['location_id','transaction_dt','customer_id_hashed'])['item_transaction_units','item_transaction_amt'].sum().reset_index()
    df_sales=df_sales.rename(columns={"item_transaction_units":"units","item_transaction_amt":"sales"})
    
    df_trans=df[['location_id','transaction_dt','transaction_id','customer_id_hashed']].drop_duplicates()
    df_trans['trans']=1
    df_trans=df_trans.groupby(['location_id','transaction_dt','customer_id_hashed'])['trans'].sum().to_frame().reset_index()
    
    df=pd.merge(df_sales,df_trans,on=["location_id",'transaction_dt','customer_id_hashed'],how="outer")
    df_agg_POS_data_2019Q3_rewards=df_agg_POS_data_2019Q3_rewards.append(df)
    
    print(datetime.datetime.now(),i_counter,df_agg_POS_data_2019Q3_rewards.shape)
    i_counter+=1
gc.collect()

2019-09-18 11:59:24.844043 0 (1312276, 6)
2019-09-18 12:07:01.660076 1 (2641439, 6)
2019-09-18 12:13:20.714391 2 (3993285, 6)
2019-09-18 12:18:58.445942 3 (5407040, 6)
2019-09-18 12:23:10.937558 4 (6796632, 6)
2019-09-18 12:27:42.354969 5 (8103909, 6)


96

In [12]:
def week_end_dt(x):
    y=datetime.datetime.strptime(x,"%Y-%m-%d").date()
    if y.weekday()==6:
        y=y+datetime.timedelta(days=6)
    else:
        y=y+datetime.timedelta(days=5-y.weekday())
    return y

In [13]:
df_agg_POS_data_2019Q3_rewards['Store_Type']=np.where(df_agg_POS_data_2019Q3_rewards['location_id']=="6990","Online",
                                                      np.where(df_agg_POS_data_2019Q3_rewards['location_id'].isin(set_SOTF_number),"SOTF","Other")
                                                     )
df_agg_POS_data_2018Q3_rewards['Store_Type']=np.where(df_agg_POS_data_2018Q3_rewards['location_id']=="6990","Online",
                                                      np.where(df_agg_POS_data_2018Q3_rewards['location_id'].isin(set_SOTF_number),"SOTF","Other")
                                                     )

In [14]:
df_week_end_dt=df_agg_POS_data_2019Q3_rewards[['transaction_dt']].drop_duplicates()
df_week_end_dt_2=df_agg_POS_data_2018Q3_rewards[['transaction_dt']].drop_duplicates()
df_week_end_dt=df_week_end_dt.append(df_week_end_dt_2)
del df_week_end_dt_2
df_week_end_dt['week_end_dt']=df_week_end_dt['transaction_dt'].apply(week_end_dt)


In [15]:
df_agg_POS_data_2019Q3_rewards=pd.merge(df_agg_POS_data_2019Q3_rewards,df_week_end_dt,on="transaction_dt",how="left")
df_agg_POS_data_2018Q3_rewards=pd.merge(df_agg_POS_data_2018Q3_rewards,df_week_end_dt,on="transaction_dt",how="left")

In [16]:
write_folder="/home/jian/Projects/Big_Lots/Analysis/2019_Q3/Projection_est_in_Q3/new_audience_view/output_"+str(datetime.datetime.now().date())+"/"
try:
    os.stat(write_folder)
except:
    os.mkdir(write_folder)

In [17]:
df_agg_POS_data_2019Q3_rewards.to_csv(write_folder+"df_agg_POS_data_2019Q3_rewards.csv",index=False)
df_agg_POS_data_2018Q3_rewards.to_csv(write_folder+"df_agg_POS_data_2018Q3_rewards.csv",index=False)

In [18]:
df_2018Q3_by_id=df_agg_POS_data_2018Q3_rewards.groupby(['customer_id_hashed','Store_Type'])['sales','units','trans'].sum().reset_index()
df_2019Q3_by_id=df_agg_POS_data_2019Q3_rewards.groupby(['customer_id_hashed','Store_Type'])['sales','units','trans'].sum().reset_index()

# df_2018Q3_by_id=df_2018Q3_by_id.pivot_table(index=['customer_id_hashed'],columns=['Store_Type'],values=['sales','units','trans'])
# df_2019Q3_by_id=df_2019Q3_by_id.pivot_table(index=['customer_id_hashed'],columns=['Store_Type'],values=['sales','units','trans'])

# df_2018Q3_by_id=df_2018Q3_by_id.fillna(0).reset_index()
# df_2019Q3_by_id=df_2019Q3_by_id.fillna(0).reset_index()

In [20]:
df_2019Q3_by_id=pd.merge(df_2019Q3_by_id,df_audience_2019Q3,on="customer_id_hashed",how="left")
df_2018Q3_by_id=pd.merge(df_2018Q3_by_id,df_audience_2019Q3,on="customer_id_hashed",how="left")

In [21]:
df_2019Q3_by_id['year']=2019
df_2018Q3_by_id['year']=2018

In [22]:
df_18_shopper=df_2018Q3_by_id[['customer_id_hashed','year']].drop_duplicates()
df_19_shopper=df_2019Q3_by_id[['customer_id_hashed','year']].drop_duplicates()

df_shopper_type=df_18_shopper.append(df_19_shopper)
df_shopper_type=df_shopper_type.groupby(['customer_id_hashed'])['year'].sum().to_frame().reset_index()
df_shopper_type['shopper_type']=np.where(df_shopper_type['year']==2019,"Shopper_19Q3_Only",
                                        np.where(df_shopper_type['year']==2018,"Shopper_18Q3_Only","Shopper_Both")
                                        )

In [23]:
del df_shopper_type['year']
df_shopper_type.head(2)

,customer_id_hashed,shopper_type
0,000000ebcf6c6a2f4302291cc9babb0760208fc683b3b5...,Shopper_19Q3_Only
1,000006ff76d207f0c8fbb0262fb87c64b31a75dc89fd07...,Shopper_Both


In [24]:
df_2019Q3_by_id=pd.merge(df_2019Q3_by_id,df_shopper_type,on="customer_id_hashed",how="left")
df_2018Q3_by_id=pd.merge(df_2018Q3_by_id,df_shopper_type,on="customer_id_hashed",how="left")

In [25]:
df_2019Q3_by_id['register_group']=np.where(df_2019Q3_by_id['customer_id_hashed'].isin(set_new_signed_ids),"Registered_After_18Q2",
                                           "Registered_Before_18Q3")
df_2018Q3_by_id['register_group']=np.where(df_2018Q3_by_id['customer_id_hashed'].isin(set_new_signed_ids),"Registered_After_18Q2",
                                           "Registered_Before_18Q3")


In [26]:
df_2018Q3_by_id.head(2)

,customer_id_hashed,Store_Type,sales,units,trans,segment,year,shopper_type,register_group
0,000006ff76d207f0c8fbb0262fb87c64b31a75dc89fd07...,Other,4.90,2,1,T_P_L_2019Q3_RFM,2018,Shopper_Both,Registered_After_18Q2
1,00000b02775629a353b0732eead2135aac6bb0502f10db...,Other,96.94,11,1,NaN,2018,Shopper_18Q3_Only,Registered_Before_18Q3


In [27]:
df_2019Q3_by_id.to_csv(write_folder+"df_2019Q3_by_id.csv",index=False)
df_2018Q3_by_id.to_csv(write_folder+"df_2018Q3_by_id.csv",index=False)

In [28]:
agg_func={"sales":"sum","units":"sum","trans":"sum","customer_id_hashed":"nunique"}
df_2019Q3_by_group_storetype=df_2019Q3_by_id.groupby(['segment','shopper_type','register_group','Store_Type'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()
df_2018Q3_by_group_storetype=df_2018Q3_by_id.groupby(['segment','shopper_type','register_group','Store_Type'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()

df_2019Q3_by_group_NoStoretype=df_2019Q3_by_id.groupby(['segment','shopper_type','register_group'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()
df_2018Q3_by_group_NoStoretype=df_2018Q3_by_id.groupby(['segment','shopper_type','register_group'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()


In [29]:
df_2019Q3_by_group_storetype['year']=2019
df_2018Q3_by_group_storetype['year']=2018

df_2019Q3_by_group_NoStoretype['year']=2019
df_2018Q3_by_group_NoStoretype['year']=2018

df_2019Q3_by_group_storetype=df_2019Q3_by_group_storetype.rename(columns={"customer_id_hashed":"shopper_id_count"})
df_2018Q3_by_group_storetype=df_2018Q3_by_group_storetype.rename(columns={"customer_id_hashed":"shopper_id_count"})
df_2019Q3_by_group_NoStoretype=df_2019Q3_by_group_NoStoretype.rename(columns={"customer_id_hashed":"shopper_id_count"})
df_2018Q3_by_group_NoStoretype=df_2018Q3_by_group_NoStoretype.rename(columns={"customer_id_hashed":"shopper_id_count"})


In [30]:
df_by_gorup_storetype=df_2019Q3_by_group_storetype.append(df_2018Q3_by_group_storetype)
df_by_gorup_NoStoretype=df_2019Q3_by_group_NoStoretype.append(df_2018Q3_by_group_NoStoretype)

In [31]:
df_by_gorup_storetype_wide_2019=df_2019Q3_by_group_storetype.pivot_table(index=["segment","shopper_type","register_group"],columns="Store_Type",values=["sales","units","trans","shopper_id_count"]).reset_index()
df_by_gorup_storetype_wide_2018=df_2018Q3_by_group_storetype.pivot_table(index=["segment","shopper_type","register_group"],columns="Store_Type",values=["sales","units","trans","shopper_id_count"]).reset_index()


In [32]:
new_col_list=[]
for col in df_by_gorup_storetype_wide_2019.columns.tolist():
    new_col="_".join(col)
    if new_col[-1]=="_":
        new_col=new_col[:-1]
    new_col_list.append(new_col)
df_by_gorup_storetype_wide_2019.columns=new_col_list

new_col_list=[]
for col in df_by_gorup_storetype_wide_2018.columns.tolist():
    new_col="_".join(col)
    if new_col[-1]=="_":
        new_col=new_col[:-1]
    new_col_list.append(new_col)
df_by_gorup_storetype_wide_2018.columns=new_col_list

In [33]:
df_by_gorup_storetype_wide_2019=df_by_gorup_storetype_wide_2019.fillna(0)
df_by_gorup_storetype_wide_2018=df_by_gorup_storetype_wide_2018.fillna(0)

In [37]:
count_aud_reg=df_audience_2019Q3.copy()

count_aud_reg['register_group']=np.where(count_aud_reg['customer_id_hashed'].isin(set_new_signed_ids),"Registered_After_18Q2",
                                           "Registered_Before_18Q3")
count_aud_reg=count_aud_reg.groupby(['segment','register_group'])['customer_id_hashed'].nunique().to_frame().reset_index()

In [38]:
writer=pd.ExcelWriter(write_folder+"BL_table_view_by_audience_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_by_gorup_storetype.to_excel(writer,"df_by_gorup_storetype",index=False)
df_by_gorup_NoStoretype.to_excel(writer,"by_gorup_NoStoretype",index=False)
df_by_gorup_storetype_wide_2019.to_excel(writer,"by_gorup_storetype_19_wide",index=False)
df_by_gorup_storetype_wide_2018.to_excel(writer,"by_gorup_storetype_18_wide",index=False)
df_audience_total_count_original.to_excel(writer,"audience_original_count",index=False)
df_audience_total_count.to_excel(writer,"audience_PS_HML_count",index=False)
count_aud_reg.to_excel(writer,"audience_Register_count",index=False)
writer.save()

# By week

In [39]:
df_agg_POS_data_2018Q3_rewards.head(2)

,location_id,transaction_dt,customer_id_hashed,units,sales,trans,Store_Type,week_end_dt
0,1,2018-08-05,03a2ad831004df2c6bf64a486e5c11841e96b5b8324fef...,8,26.78,1,SOTF,2018-08-11
1,1,2018-08-05,074be6c6b65fef7dbf023a10df96ec80d3463b2c1067ac...,0,0.00,2,SOTF,2018-08-11


In [42]:
df_2018Q3_by_id_week=df_agg_POS_data_2018Q3_rewards.groupby(['customer_id_hashed','Store_Type','week_end_dt'])['sales','units','trans'].sum().reset_index()
df_2019Q3_by_id_week=df_agg_POS_data_2019Q3_rewards.groupby(['customer_id_hashed','Store_Type','week_end_dt'])['sales','units','trans'].sum().reset_index()


In [43]:
df_2018Q3_by_id_week=pd.merge(df_2018Q3_by_id_week,df_audience_2019Q3,on="customer_id_hashed",how="left")
df_2019Q3_by_id_week=pd.merge(df_2019Q3_by_id_week,df_audience_2019Q3,on="customer_id_hashed",how="left")

In [44]:
df_2018Q3_by_id_week['year']=2018
df_2019Q3_by_id_week['year']=2019

In [45]:
# df_shopper_type keep the same for the 6 weeks defination

df_2018Q3_by_id_week=pd.merge(df_2018Q3_by_id_week,df_shopper_type,on="customer_id_hashed",how="left")
df_2019Q3_by_id_week=pd.merge(df_2019Q3_by_id_week,df_shopper_type,on="customer_id_hashed",how="left")

In [46]:
df_2018Q3_by_id_week['register_group']=np.where(df_2018Q3_by_id_week['customer_id_hashed'].isin(set_new_signed_ids),"Registered_After_18Q2",
                                           "Registered_Before_18Q3")
df_2019Q3_by_id_week['register_group']=np.where(df_2019Q3_by_id_week['customer_id_hashed'].isin(set_new_signed_ids),"Registered_After_18Q2",
                                           "Registered_Before_18Q3")


In [47]:
df_2018Q3_by_id_week.to_csv(write_folder+"df_2018Q3_by_id_week.csv",index=False)
df_2019Q3_by_id_week.to_csv(write_folder+"df_2019Q3_by_id_week.csv",index=False)

In [49]:
df_2018Q3_by_id_week.head(2)

,customer_id_hashed,Store_Type,week_end_dt,sales,units,trans,segment,year,shopper_type,register_group
0,000006ff76d207f0c8fbb0262fb87c64b31a75dc89fd07...,Other,2018-08-18,4.90,2,1,T_P_L_2019Q3_RFM,2018,Shopper_Both,Registered_After_18Q2
1,00000b02775629a353b0732eead2135aac6bb0502f10db...,Other,2018-08-18,96.94,11,1,NaN,2018,Shopper_18Q3_Only,Registered_Before_18Q3


In [50]:
agg_func={"sales":"sum","units":"sum","trans":"sum","customer_id_hashed":"nunique"}
df_2018Q3_by_group_storetype_week=df_2018Q3_by_id_week.groupby(['week_end_dt','segment','shopper_type','register_group','Store_Type'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()
df_2019Q3_by_group_storetype_week=df_2019Q3_by_id_week.groupby(['week_end_dt','segment','shopper_type','register_group','Store_Type'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()

df_2018Q3_by_group_NoStoretype_week=df_2018Q3_by_id_week.groupby(['week_end_dt','segment','shopper_type','register_group'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()
df_2019Q3_by_group_NoStoretype_week=df_2019Q3_by_id_week.groupby(['week_end_dt','segment','shopper_type','register_group'])['sales','units','trans','customer_id_hashed'].agg(agg_func).reset_index()


In [51]:
df_2019Q3_by_group_storetype_week['year']=2019
df_2018Q3_by_group_storetype_week['year']=2018

df_2019Q3_by_group_NoStoretype_week['year']=2019
df_2018Q3_by_group_NoStoretype_week['year']=2018

df_2019Q3_by_group_storetype_week=df_2019Q3_by_group_storetype_week.rename(columns={"customer_id_hashed":"shopper_id_count"})
df_2018Q3_by_group_storetype_week=df_2018Q3_by_group_storetype_week.rename(columns={"customer_id_hashed":"shopper_id_count"})
df_2019Q3_by_group_NoStoretype_week=df_2019Q3_by_group_NoStoretype_week.rename(columns={"customer_id_hashed":"shopper_id_count"})
df_2018Q3_by_group_NoStoretype_week=df_2018Q3_by_group_NoStoretype_week.rename(columns={"customer_id_hashed":"shopper_id_count"})


In [52]:
df_by_gorup_storetype_week=df_2019Q3_by_group_storetype_week.append(df_2018Q3_by_group_storetype_week)
df_by_gorup_NoStoretype_week=df_2019Q3_by_group_NoStoretype_week.append(df_2018Q3_by_group_NoStoretype_week)

In [53]:
df_by_gorup_storetype_wide_2019_week=df_2019Q3_by_group_storetype_week.pivot_table(index=["week_end_dt","segment","shopper_type","register_group"],columns="Store_Type",values=["sales","units","trans","shopper_id_count"]).reset_index()
df_by_gorup_storetype_wide_2018_week=df_2018Q3_by_group_storetype_week.pivot_table(index=["week_end_dt","segment","shopper_type","register_group"],columns="Store_Type",values=["sales","units","trans","shopper_id_count"]).reset_index()


In [54]:
new_col_list=[]
for col in df_by_gorup_storetype_wide_2019_week.columns.tolist():
    new_col="_".join(col)
    if new_col[-1]=="_":
        new_col=new_col[:-1]
    new_col_list.append(new_col)
df_by_gorup_storetype_wide_2019_week.columns=new_col_list

new_col_list=[]
for col in df_by_gorup_storetype_wide_2018_week.columns.tolist():
    new_col="_".join(col)
    if new_col[-1]=="_":
        new_col=new_col[:-1]
    new_col_list.append(new_col)
df_by_gorup_storetype_wide_2018_week.columns=new_col_list

In [55]:
df_by_gorup_storetype_wide_2019_week=df_by_gorup_storetype_wide_2019_week.fillna(0)
df_by_gorup_storetype_wide_2018_week=df_by_gorup_storetype_wide_2018_week.fillna(0)

In [60]:
writer=pd.ExcelWriter(write_folder+"BL_table_view_by_week_audience_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_by_gorup_storetype_week.to_excel(writer,"View_Week_gorup_storetype",index=False)
df_by_gorup_NoStoretype_week.to_excel(writer,"View_Week_gorup_NoStoretype",index=False)
df_by_gorup_storetype_wide_2019_week.to_excel(writer,"View_W_gorup_storetype_19_wide",index=False)
df_by_gorup_storetype_wide_2018_week.to_excel(writer,"View_W_gorup_storetype_18_wide",index=False)
df_audience_total_count_original.to_excel(writer,"audience_original_count",index=False)
df_audience_total_count.to_excel(writer,"audience_PS_HML_count",index=False)
count_aud_reg.to_excel(writer,"audience_Register_count",index=False)
writer.save()

In [59]:
# Note how to difine the new sign ups
